In [1]:
# Instalar las librerías necesarias para ejecutar el script
%pip install openai
%pip install llama-index
%pip install pypdf
%pip install langchain
%pip install "unstructured[all-docs]"
%pip install faiss-gpu
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached faiss-gpu-1.7.1.post2.tar.gz (40 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz has inconsistent name: expected 'faiss-gpu', but metadata has 'faiss-cpu'
  Using cached faiss-gpu-1.7.1.post1.tar.gz (41 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/39/8d/b6

In [5]:
pip install google.cloud.storage

  Obtaining dependency information for google.cloud.storage from https://files.pythonhosted.org/packages/88/14/c9d4faae7ea4bff4405152cbc762b61100aa6949273b4eb3203d23308670/google_cloud_storage-2.10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-core<3.0dev,>=2.3.0 from https://files.pythonhosted.org/packages/a2/40/02045f776fdb6e44194f34b6375a26ce8a61bd9bd03cd8930ed91cf51a62/google_cloud_core-2.3.3-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 572.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 1.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os

#os.environ["OPENAI_API_KEY"] =
openai.api_key = "xxx"

In [4]:
from langchain.document_loaders import UnstructuredFileLoader
from google.cloud import storage
from google.oauth2 import service_account
import tempfile
import os

def extract_file_content(file_path, max_characters):
    loader = UnstructuredFileLoader(file_path)
    documents = loader.load()
    documents_content = '\n'.join(doc.page_content for doc in documents)
    return documents_content[:max_characters]

def download_file_from_gcs(bucket_name, file_name, service_account_json):
    """Downloads a file from Google Cloud Storage and returns its content as bytes.

    Args:
        bucket_name (str): The name of the Google Cloud Storage bucket.
        file_name (str): The name of the file to download.
        service_account_json (str): The path to the service account JSON file.

    Returns:
        str: The local path to the downloaded file.
    """
    # Load the service account credentials from the JSON file.
    credentials = service_account.Credentials.from_service_account_file(service_account_json)

    # Create a storage client with the loaded credentials.
    client = storage.Client(credentials=credentials)

    bucket = client.bucket(bucket_name)
    file_object = bucket.blob(file_name)

    # Check if the object exists and fetch its content.
    if not file_object.exists():
        raise FileNotFoundError(f"The object '{file_name}' does not exist in the bucket '{bucket_name}'.")

    # Create a temporary directory to store the downloaded file.
    temp_dir = tempfile.mkdtemp()
    local_file_path = os.path.join(temp_dir, file_name)

    # Download the file to the temporary directory.
    file_object.download_to_filename(local_file_path)

    return local_file_path

if __name__ == "__main__":
    bucket_name = "clausulados"
    file_name = "Prot_tarj.pdf"
    service_account_json = "service_account_key.json"  # Replace with the actual path to your service account JSON file.
    max_characters = 400

    # Download the file from Google Cloud Storage
    local_file_path = download_file_from_gcs(bucket_name, file_name, service_account_json)

    # Extract the content using the UnstructuredFileLoader and limit to the first 400 characters
    extracted_content = extract_file_content(local_file_path, max_characters)

    # Now 'extracted_content' contains the first 400 characters of the text extracted from the PDF.
    print(f"First {max_characters} Characters of the Document: \n{extracted_content}...")


First 400 Characters of the Document: 
SEGURO DE PROTECCIÓN PATRIMONIAL TARJETAHABIENTES 27/09/2021-1327-P-06-RC-0000000000046-0R00

CONDICIONES GENERALES

ASEGURADO Es el titular de la(s) tarjetas(s) débito o crédito expedidos por el B a n c o D a v i v i e n d a y q u i e n voluntariamente se adhiere a la póliza.

le Seguros Comerciales Bolívar en adelante reconocerá al o beneficiario, el pago del valor asegurado, hasta el número y v...
